# Validation
In this notebook we will validate model datasets against the GridClim product, or any other gridded observations.
There are four metrics available in `attribution.validation`
- The average anomaly
- The monthly average anomaly
- A seasonal correlation index
- A spatial correlation index

In [1]:
import glob
import os
from importlib import reload

import attribution.validation
import dask
import iris
import iris.plot as iplt
import numpy as np
import pandas as pd
from attribution.config import init_config
from dask.distributed import Client
from matplotlib import pyplot as plt

/home/sm_erhol/dev/attribution/attribution/bootstrap.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
client = Client()

In [3]:
CFG = init_config()

In [4]:
# Where do we store the data?
project_folder = CFG["paths"]["project_folder"]

In [5]:
# We are investigating multiple variables, have to select one.
variables = CFG["variable"]

In [6]:
# Which variable will we use?
variable = variables[0]
print(variable)

tasmax


In [7]:
gc_path = glob.glob(os.path.join(project_folder,  f"{variable}*GridClim*.nc"))
cordex_path = glob.glob(os.path.join(project_folder, f"{variable}*CORDEX*.nc"))

In [8]:
# GridClim
gc_cube = iris.load_cube(gc_path)
# Cordex
cordex_cube = iris.load_cube(cordex_path)

/tmp/ipykernel_11352/298512951.py:2: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  gc_cube = iris.load_cube(gc_path)
/tmp/ipykernel_11352/298512951.py:4: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  cordex_cube = iris.load_cube(cordex_path)


## Average (daily) anomaly
This calculates the average day anomaly between the model and reference dataset.

In [9]:
average_anomaly = attribution.validation.average_anomaly(cordex_cube, gc_cube)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


Note that since the cube data is lazy, the above step doesn't actually perform the calculation.
This happens first when we need the data, like below.

In [10]:
# Print the average anomaly as a percentage.
average_anomaly.data * 100

masked_array(data=[0.02470068, 0.04263533, 0.12229737, 0.05447186,
                   0.14864741, 0.14248091, 0.12344603, 0.12869356,
                   0.09480892, 0.06586543, 0.12809428, 0.20485643,
                   0.06805379, 0.16460385, 0.09835906, 0.03320235,
                   0.03475651, 0.03864188, 0.03343472, 0.15728814,
                   0.18471139, 0.02701622, 0.05009516, 0.12289297,
                   0.10228395, 0.02486696, 0.02943476, 0.07067249,
                   0.03090618, 0.02507413, 0.0323759 , 0.08691694,
                   0.0677028 , 0.03682929, 0.06210044, 0.04367879,
                   0.08495711, 0.03288937, 0.05240937, 0.04021105,
                   0.03507161, 0.24875904, 0.2307421 , 0.36306812,
                   0.05022454, 0.22094794, 0.20978311, 0.04064428,
                   0.17193602, 0.27398672, 0.19574699, 0.19446234,
                   0.22157214, 0.41526384, 0.19977783, 0.2994604 ,
                   0.02719066, 0.2315728 , 0.15503526, 0.21517

## Average monthly anomaly


In [11]:
average_monthly_anomaly = attribution.validation.average_monthly_anomaly(
    cordex_cube, gc_cube
)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [12]:
# The average seasonal anomaly as a percentage.
average_monthly_anomaly.data * 100

masked_array(data=[0.13056592, 0.1319187 , 0.20149965, 0.16116541,
                   0.19489767, 0.19587427, 0.15264203, 0.20121151,
                   0.16618229, 0.15462652, 0.20884416, 0.25902771,
                   0.15249099, 0.23122849, 0.16736746, 0.10953116,
                   0.14734591, 0.14682785, 0.14110017, 0.2216421 ,
                   0.26080939, 0.1342513 , 0.13394846, 0.20738143,
                   0.17833268, 0.15933751, 0.1446479 , 0.20575523,
                   0.13052691, 0.13773802, 0.18052141, 0.14828238,
                   0.12781301, 0.12003914, 0.15005886, 0.13960783,
                   0.12628491, 0.14096685, 0.1504453 , 0.16721578,
                   0.11289474, 0.26159789, 0.25509457, 0.37183853,
                   0.11542072, 0.24138795, 0.22745375, 0.11085283,
                   0.18337672, 0.27733798, 0.21827833, 0.22923265,
                   0.23916782, 0.42371432, 0.22209226, 0.32332459,
                   0.10094573, 0.29025179, 0.23577885, 0.29296

## Seasonal correlation index

There are essentially two indices we can calculate here, depending on how hard we want to test the model.
If we want to test the models ability to simulate the average year we don't need to set the arg `climatological` since this is `True` by default.
This will correlate the average annual cycle for the model(s) and observations in each grid point.
Doing this checks how well the model captures the average annual cycle for each grid point.

By setting `climatological=False` every annual cycle is instead correlated i.e. monthly values.
This is a much tougher metric, and more computationally demanding.

There are also two version of this metric, one which follows the definition in the report "Das Bayerische Klimaprojektionensemble", and one which more directly follow the Kling-Gupta efficiency.
This is switched with the `kge` kwarg.

In [13]:
seasonality_index = attribution.validation.seasonality_index(
    cordex_cube, gc_cube, kge=False
)
seasonality_index_kge = attribution.validation.seasonality_index(
    cordex_cube, gc_cube, kge=True
)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarni

In [14]:
seasonality_index.data

masked_array(data=[0.97569263, 0.97623617, 0.98479301, 0.97211201,
                   0.99004929, 0.99006441, 0.98985097, 0.9814479 ,
                   0.98082035, 0.98576553, 0.98959309, 0.93347707,
                   0.98768412, 0.98801475, 0.96438424, 0.98317449,
                   0.98889188, 0.9821033 , 0.98854337, 0.98841832,
                   0.9331463 , 0.99163913, 0.987898  , 0.98571287,
                   0.96459438, 0.94194575, 0.95663277, 0.94467288,
                   0.95919937, 0.95668494, 0.98632865, 0.96458347,
                   0.98757689, 0.98805588, 0.98736504, 0.98556002,
                   0.988899  , 0.98995162, 0.98828807, 0.95856509,
                   0.97957098, 0.98905746, 0.98419373, 0.93335635,
                   0.98178652, 0.98636331, 0.98879605, 0.99030189,
                   0.99146144, 0.99057369, 0.99232804, 0.95650793,
                   0.98961368, 0.92579013, 0.98072681, 0.91622734,
                   0.98852707, 0.93093013, 0.93893655, 0.93238

In [15]:
seasonality_index_kge.data

masked_array(data=[0.8912327 , 0.88762622, 0.90063503, 0.92876504,
                   0.92344917, 0.92561695, 0.91913462, 0.91688976,
                   0.90041867, 0.90028122, 0.86545323, 0.93340284,
                   0.88379059, 0.90553359, 0.92309383, 0.89762842,
                   0.87270125, 0.86427852, 0.88671199, 0.89999452,
                   0.9344564 , 0.88030585, 0.87258724, 0.89952491,
                   0.94100684, 0.89983968, 0.91769379, 0.91660974,
                   0.92371404, 0.92611955, 0.91143518, 0.87630046,
                   0.89552907, 0.88388847, 0.90221956, 0.89004341,
                   0.88308905, 0.87770513, 0.88851653, 0.87770899,
                   0.91445767, 0.89806974, 0.88320655, 0.86772188,
                   0.89194178, 0.90257284, 0.89492251, 0.89323544,
                   0.90339652, 0.86265304, 0.9090939 , 0.89585076,
                   0.88582764, 0.88191578, 0.86748168, 0.82958174,
                   0.89508619, 0.85306983, 0.88267102, 0.88739

## Spatial correlation
Measures the spatial correlation between model and reference dataset.

In [16]:
pattern_index = attribution.validation.pattern_index(cordex_cube, gc_cube)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'grid_latitude'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2223: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'grid_longitude'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2217: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2217: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packa

In [17]:
pattern_index.data

masked_array(data=[0.94427043, 0.9057405 , 0.9228785 , 0.94113064,
                   0.90813535, 0.8954709 , 0.9016418 , 0.92411685,
                   0.94117486, 0.93929374, 0.9364288 , 0.9362363 ,
                   0.9141613 , 0.924579  , 0.925389  , 0.8934054 ,
                   0.94370717, 0.92170775, 0.92711234, 0.92962706,
                   0.9260852 , 0.9499937 , 0.921403  , 0.91723645,
                   0.9235031 , 0.93570113, 0.92121816, 0.9421294 ,
                   0.9406837 , 0.94300026, 0.9309614 , 0.9080189 ,
                   0.935829  , 0.9396764 , 0.91469586, 0.91022944,
                   0.9376457 , 0.9252795 , 0.93925136, 0.9426521 ,
                   0.93255454, 0.9418003 , 0.9431238 , 0.94470143,
                   0.93974   , 0.9395513 , 0.93843585, 0.9069074 ,
                   0.94216686, 0.92282313, 0.93251526, 0.9451926 ,
                   0.9120572 , 0.9185193 , 0.93010825, 0.93808836,
                   0.92175096, 0.9397377 , 0.9424222 , 0.89659

## Homogenise the scores
Mapping the index to scores between 0 and 10.
The bins used are based on values from the paper/report on the Bavarian climate projection ensemble.

In [18]:
# Scores for index 1 and 2
indx_1_scores = attribution.validation.get_scores(
    average_anomaly.data * 100,
    bins=np.arange(5, 51, 5),
    score_bins=np.arange(10, -1, -1),
)
indx_2_scores = attribution.validation.get_scores(
    average_monthly_anomaly.data * 100,
    bins=np.arange(5, 51, 5),
    score_bins=np.arange(10, -1, -1),
)

In [19]:
# Scores for index 3 and 4
indx_3_scores = attribution.validation.get_scores(
    seasonality_index.data,
    bins=np.arange(0.92, 0.19, -0.08),
    score_bins=np.arange(10, -1, -1),
)
indx_4_scores = attribution.validation.get_scores(
    pattern_index.data,
    bins=np.arange(0.92, 0.19, -0.08),
    score_bins=np.arange(10, -1, -1),
)

In [20]:
# Stack the scores
data = np.stack(
    [
        indx_1_scores,
        indx_2_scores,
        indx_3_scores,
        indx_4_scores,
    ],
    axis=1,
)

Create a `pandas.Dataframe` of the scores

In [21]:
# Create a dataframe of the scores.
scores_df = pd.DataFrame(data=data, columns=["Idx_1", "Idx_2", "Idx_3", "Idx_4"])
# Add the ensemble id
scores_df["ensemble_id"] = cordex_cube.coord("ensemble_id").points
scores_df = scores_df[["ensemble_id", "Idx_1", "Idx_2", "Idx_3", "Idx_4"]]

In [22]:
# Add a sum of the scores.
scores_df["Sum_scores"] = scores_df.sum(axis=1, numeric_only=True)

In [23]:
scores_df.min()

ensemble_id    CCCma-CanESM2--CLMcom-CCLM4-8-17--r1i1p1
Idx_1                                                10
Idx_2                                                10
Idx_3                                                 9
Idx_4                                                 9
Sum_scores                                           39
dtype: object

With these scores we can evaluate if there are any models that should be excluded from the rest of the study.
In this particular case, the lowest total score is 32, which is still acceptable, and we don't exclude any models based on this step.

In [26]:
# Save the scores to a csv.
scores_df.to_csv(os.path.join(project_folder, "scores.csv"))

In [ ]:
client.shutdown()

## Next step

[Attribution](4_attribution.ipynb)